# Ex8 - Transformação de visão e de projeção

Nesta atividade, vocês vão exercitar os conceitos de transformação de visão e de projeção. Para evitar erros de execução, utilize apenas uma célula de código para cada parte desta atividade. O teste de profundidade deve estar ativado em todos os exercícios.

### Parte 1 - Transformação de visão

Continuando com a construção do sistema solar da Atividade 7 ([EA979A_Ex07_Modelos_e_transformacoes-Gabarito](EA979A_Ex07_Modelos_e_transformacoes-Gabarito.ipynb)), adicione a renderização de Júpiter ao programa. As informações dele são fornecidas no código abaixo. Você vai perceber que o novo planeta adicionado não cabe na tela. Então, é preciso realizar algumas transformações para visualizá-lo. Para isso, crie uma matriz de visão e trate os eventos das setas do teclado para deslocar, simultaneamente, os parâmetros desta matriz (o ponto de visão e a posição da câmera) na direção da tecla pressionada. Ou seja, as setas esquerda e direita deslocam estes parâmetros no eixo x, e as setas 'para cima' e 'para baixo' deslocam eles no eixo y. Além disso, utilize as teclas + e - para alterar uma transformação global que, respectivamente, aumenta e diminui o tamanho de todos os objetos renderizados. A imagem abaixo mostram a aparência esperada com a adição de Júpiter. Comente o resultado obtido ao mudar os parâmetros de visão. O notebook ([29_Transformacao_visao](29_Transformacao_visao.ipynb)) exemplifica algumas formas de como criar e utilizar a matriz de visão.


<td><img src='cg/images/ex8_solar_system_3.png' style="width:300px">

In [16]:
import glm
import time
import math
import numpy as np
import OpenGL.GL as gl
from PyQt5 import QtOpenGL, QtCore
from PyQt5.QtCore import QCoreApplication
from PyQt5.QtWidgets import QApplication

from cg.shader_programs.SimpleShaderProgram_v2 import SimpleShaderProgram
from cg.renderers.ModelRenderer_v2 import ModelRenderer
from cg.models.SphereMesh_v1 import SphereMesh
from cg.models.OrbitPolygon import OrbitPolygon

AU = 1.496E8
planet_scale_factor = 1500.0
moon_scale_factor = planet_scale_factor * 0.05
sun_scale_factor = 50

# classe para armazenar as informações do Sol
class SunInfo:
    def __init__(self, color, diameter):
        
        self.color = color
        self.diameter = (diameter / AU) * sun_scale_factor
        self.radius = self.diameter / 2.0

# classe para armazenar as informações dos planetas
class PlanetInfo:
    def __init__(self, color, diameter, orbitalPeriod, orbitalInclination, orbitSemiMajorAxis, orbitSemiMinorAxis, orbitFocus):
        
        earth_days = 365.2
        
        self.color = color
        self.diameter = (diameter / AU) * planet_scale_factor
        self.radius = self.diameter / 2.0
        self.velocity = earth_days / orbitalPeriod
        
        self.orbitalPeriod = orbitalPeriod
        self.orbitalInclination = orbitalInclination 
        self.orbitSemiMajorAxis = orbitSemiMajorAxis
        self.orbitSemiMinorAxis = orbitSemiMinorAxis
        self.orbitFocus = orbitFocus

# cria um dicionário contendo as informações dos planetas       
PlanetarySheet = {
       'sun': SunInfo(np.array([1.00, 1.00, 0.00, 1.0]), 1391900),
       'earth': PlanetInfo(np.array([0.61, 0.79, 0.37, 1.0]), 12742, 365.2, 1.57, 1.0027, 1.0025, 0.0167),
       'moon': PlanetInfo(np.array([0.50, 0.50, 0.50, 1.0]), 3475, 27.3, 5.1, 0.0025718 * moon_scale_factor, 0.0021479 * moon_scale_factor, 0.00014537),
       'jupiter': PlanetInfo(np.array([0.83, 0.67, 0.53, 1.0]), 142984, 4331, 0.32, 5.2073, 5.2010, 0.2520)
       }

# classe para armazenar as informações gráficas em um único objeto
class GraphicObject:
    def __init__(self):
        self.model = None
        self.renderer = None
        self.modelMatrix = glm.mat4()
        self.color = np.array([1.0, 1.0, 1.0, 1.0])
        
class MyWidget(QtOpenGL.QGLWidget):

    def initializeGL(self):
        
        self.planetName = ['earth', 'moon', 'jupiter',]
        self.orbits = {}
        self.planets = {}
        
        self.cameraPos   = glm.vec3(0.0, 0.0, 0.0)
        self.cameraFront = glm.vec3(0.0, 0.0, -1.0)
        self.cameraUp    = glm.vec3(0.0, 1.0,  0.0)
        
        self.viewMatrix = glm.lookAt(self.cameraPos, self.cameraFront, self.cameraUp)
        
        # define uma transformação global
        # os objetos no tamanho original não cabem no volume de renderização. 
        # Então eles precisam ser escalado para um tamanho menor
        self.globalScale = 0.8
        self.globalTransform = glm.scale(glm.mat4(), glm.vec3(self.globalScale, self.globalScale, self.globalScale))
        
        # cria um shader program simples   
        # ativa o shader programa para permitir configurar uma cor única para todos os vértices
        self.shaderProgram = SimpleShaderProgram()
        self.shaderProgram.bind()
        self.shaderProgram.useUniformColor(True)
        self.shaderProgram.release()
        
        position_loc = self.shaderProgram.getVertexPositionLoc()

        # cria uma malhar de triângulos esférica comum para objetos esféricos
        sphere_mesh = SphereMesh(1.0, 30, 30)
        sphere_renderer = ModelRenderer(sphere_mesh.getVertexPositions(), vertex_indices=sphere_mesh.getVertexIndices())
        sphere_renderer.setVertexPositionLoc(position_loc)
        
        # laço para cria os objetos onde as informações gráficas das órbita são armazenadas
        for name in self.planetName:
            planet_info = PlanetarySheet[name]
            self.orbits[name] = GraphicObject()
            self.orbits[name].model = OrbitPolygon(planet_info.orbitSemiMajorAxis, planet_info.orbitSemiMinorAxis, planet_info.orbitFocus, 50)
            self.orbits[name].renderer = ModelRenderer(self.orbits[name].model.getVertexPositions(), primitive=ModelRenderer.LINE_LOOP)
            self.orbits[name].renderer.setVertexPositionLoc(position_loc)
            self.orbits[name].color = planet_info.color
        
        # laço para cria os objeto onde as informações gráficas dos planetas são armazenadas
        for name in self.planetName:
            self.planets[name] = GraphicObject()
            self.planets[name].model = sphere_mesh
            self.planets[name].renderer = sphere_renderer
            self.planets[name].color = PlanetarySheet[name].color
        
        # cria um objeto para armazenar as informações gráficas do Sol
        self.sun = GraphicObject()
        self.sun.model = sphere_mesh
        self.sun.renderer = sphere_renderer
        self.sun.color = PlanetarySheet['sun'].color
        self.sun.modelMatrix = glm.scale(glm.mat4(), glm.vec3(PlanetarySheet['sun'].radius, PlanetarySheet['sun'].radius, PlanetarySheet['sun'].radius))
        
        # armazena o momento que o programa começou
        self.startTime = time.time()
    
    def paintGL(self):
        
        # configura a cor de background
        gl.glClearColor(1, 1, 1, 1)
        
        # limpa o background com a cor especificada
        gl.glClear(gl.GL_COLOR_BUFFER_BIT | gl.GL_DEPTH_BUFFER_BIT)
        
        # ativa o teste de profunidade e o recurso de culling
        gl.glEnable(gl.GL_DEPTH_TEST);
        gl.glEnable(gl.GL_CULL_FACE)
        
        # calcula o tempo de execução do programa
        self.currentTime = time.time()
        time_elapsed = self.currentTime - self.startTime
        
        # atualiza a posição dos objetos gráficos
        self.updatePositions(time_elapsed)
        
        # ativa o shader program que será executado pela GPU
        self.shaderProgram.bind()
        
        # renderiza o Sol
        self.renderObject(self.sun)
        
        # renderiza os objetos com as posições atualizadas
        self.renderPlanets()        
        self.renderOrbits()
        
        # desativa o shader program
        self.shaderProgram.release()
        
        # solicita que o método paintGL seja chamado novamente
        self.update()
    
    def resizeGL(self, width, height):
        
        # atualiza a área de renderização para ser a janela inteira
        gl.glViewport(0, 0, width, height)
    
    def updatePositions(self, time_elapsed):
        
        angle = time_elapsed * 2 * math.pi * 0.05

        self.updatePlanetPosition(angle)
        self.updateMoonOrbitPosition(angle)
        self.updateMoonPosition(angle)
        
    def updatePlanetPosition(self, angle):
        
        # atualiza a transformação de todos os planetas
        for name in self.planetName:
            
            # recupera a posição do planeta
            # e cria uma matriz de translação para levar o planeta para a nova posição
            planet_info = PlanetarySheet[name]
            x, y = self.orbits[name].model.getPositionAt(angle * planet_info.velocity)
            transl_planet_pos = glm.translate(glm.mat4(), glm.vec3(x, y, 0.0))
            
            # cria uma matriz de escala para deixar o planeta com o tamanho correto
            scale_planet_size = glm.scale(glm.mat4(), glm.vec3(planet_info.radius, planet_info.radius, planet_info.radius))
            
            # calcula a matriz de transformação final do planeta
            self.planets[name].modelMatrix = transl_planet_pos * scale_planet_size
        
    def updateMoonOrbitPosition(self, angle):
        
        # recupera a posição da Terra na órbita
        # e cria uma matriz de translação para levar a órbita da lua para a posição da Terra
        x, y = self.orbits['earth'].model.getPositionAt(angle * PlanetarySheet['earth'].velocity)
        trans_earth_pos = glm.translate(glm.mat4(), glm.vec3(x, y, 0.0))
        
        # calcula a matriz de transformação final da órbita da lua
        self.orbits['moon'].modelMatrix = trans_earth_pos
    
    def updateMoonPosition(self, angle):
        
        # recupera a posição da lua na órbita
        # e cria uma matriz de translação para levar a lua para a nova posição
        x, y = self.orbits['moon'].model.getPositionAt(angle * PlanetarySheet['moon'].velocity)
        transl_moon_pos = glm.translate(glm.mat4(), glm.vec3(x, y, 0.0))
        
        # recupera a posição da Terra na órbita
        # e cria uma matriz de translação para levar a lua para a posição da Terra
        x, y = self.orbits['earth'].model.getPositionAt(angle * PlanetarySheet['earth'].velocity)
        transl_earth_pos = glm.translate(glm.mat4(), glm.vec3(x, y, 0.0))
        
        # cria uma matriz de escala para deixar a lua com o tamanho correto
        scale_moon_size = glm.scale(glm.mat4(), glm.vec3(PlanetarySheet['moon'].radius, PlanetarySheet['moon'].radius, PlanetarySheet['moon'].radius))

        # calcula a matriz de transformação final da lua
        self.planets['moon'].modelMatrix = transl_earth_pos * transl_moon_pos * scale_moon_size
        
    def renderObject(self, graphic_object):
        
        # atualiza o shader com a transformação dos objetos
        self.shaderProgram.setUniformMVPMatrix(self.viewMatrix * self.globalTransform * graphic_object.modelMatrix)
        self.shaderProgram.setUniformColor(graphic_object.color)
        
        # renderiza o objeto
        graphic_object.renderer.render()
        
    def renderPlanets(self):
        
        # laço para renderizar todos os planetas
        for name in self.planetName:
            self.renderObject(self.planets[name])
            
    def renderOrbits(self):
        
        # laço para renderizar todas as órbitas
        for name in self.planetName:
            self.renderObject(self.orbits[name])      
    
    def keyPressEvent(self, event):
        super(MyWidget, self).keyPressEvent(event)
        
        # passo 
        step = 0.1
        
        # verifica se foi pressionada a tecla de seta 'para cima'
        if event.key() == QtCore.Qt.Key_Up:
            self.cameraPos.y += step
            self.cameraFront.y += step

        # verifica se foi pressionada a tecla de seta 'para baixo'
        elif event.key() == QtCore.Qt.Key_Down:
            self.cameraPos.y -= step
            self.cameraFront.y -= step
        
        # verifica se foi pressionada a tecla de seta 'para esquerda'
        elif event.key() == QtCore.Qt.Key_Left:
            self.cameraPos.x -= step
            self.cameraFront.x -= step
        
        # verifica se foi pressionada a tecla de seta 'para direita'
        elif event.key() == QtCore.Qt.Key_Right:
            self.cameraPos.x += step
            self.cameraFront.x += step
        
        # verifica se foi pressionada a tecla de seta '-'
        elif event.key() == QtCore.Qt.Key_Minus:
            if(self.globalScale > 0.01):
                self.globalScale -= 0.01
        
        # verifica se foi pressionada a tecla de seta '+'
        elif event.key() == QtCore.Qt.Key_Plus:
            self.globalScale += 0.01
        
        # atualiza a matriz de visão e global com os novos parâmetros
        self.viewMatrix = glm.lookAt(self.cameraPos, self.cameraFront, self.cameraUp)
        self.globalTransform = glm.scale(glm.mat4(), glm.vec3(self.globalScale, self.globalScale, self.globalScale))
        
def main():
    import sys

    app = QCoreApplication.instance()
    if app is None:
        app = QApplication(sys.argv)

    glformat = QtOpenGL.QGLFormat()
    glformat.setVersion(3, 3)
    glformat.setProfile(QtOpenGL.QGLFormat.CoreProfile)
    
    w = MyWidget(glformat)
    w.resize(900, 900)
    w.setWindowTitle('Solar System')
    w.show()
    
    output = app.exec_()
    sys.exit(output)
    
if __name__ == '__main__':
    main()

SystemExit: 0

C:\Users\Rapha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Parte 2 - Transformações de projeção

Adicione na renderização realizada na Parte 1 dois quadrados com cores diferentes ([27_Quadrado_com_transformacoes](27_Quadrado_com_transformacoes.ipynb)). O primeiro tem tamanho 25x25 e deve ser transladado para a posição (0, 0, -15). O segundo tem tamanho 30x30 e deve ser transladado para a posição (0, 0, -20). Renderize esta cena de duas formas. Uma delas utilizando uma matriz de projeção ortográfica e a outra utilizando uma matriz de projeção perspectiva. Realiza os ajustes necessários somente nos parâmetros das matrizes de visão e de projeção destas duas cenas para cada cena renderizar uma imagem semelhante à imagem abaixo (com os quadrados centrados na tela e com as bordas dos quadrados longe das bordas da janela). O ponto de visão está fixo na posição (0, 0, -1). Diferente da Parte 1, a matriz de transformação global deve ser uma matriz identidade e não há tratamento de eventos do teclado. As duas cenas podem ser renderizadas por um único programa através de múltiplos viewports ou por dois programas distintos. Comente como você chegou nos valores dos parâmetros das matrizes para obter as cenas pedidas. Os notebooks ([30_Transformacao_projecao_ortogonal](30_Transformacao_projecao_ortogonal.ipynb)) e ([31_Transformacao_projecao_perspectiva](31_Transformacao_projecao_perspectiva.ipynb)) exemplificam algumas formas de como criar e utilizar as matrizes de projeção.


<td><img src='cg/images/ex8_solar_system_4.png' style="width:400px">

### Parte 3 - Transformações de visão e de projeção

Utilizando o código da Parte 2, trate os eventos das setas do teclado para alterar apenas a posição da matriz de visão (o ponto de visão está fixo na posição (0, 0, -1)). Desloque o parâmetro de posição da câmera desta matriz na direção da tecla pressionada. Ou seja, as setas esquerda e direita deslocam no eixo x, e as setas 'para cima' e 'para baixo' deslocam no eixo y. Além disso, utilize as teclas + e - para, respectivamente, incrementar e decrementar a componente z da posição da câmera. As imagens abaixo mostram o efeito obtido quando a posição da câmera é transladada para a esquerda. Comente as diferenças entre as imagens renderizadas com a projeção perspectiva e com a projeção ortográfica ao mudar a posição da câmera. Os notebooks ([29_Transformacao_visao](29_Transformacao_visao.ipynb)), ([30_Transformacao_projecao_ortogonal](30_Transformacao_projecao_ortogonal.ipynb)) e ([31_Transformacao_projecao_perspectiva](31_Transformacao_projecao_perspectiva.ipynb)) exemplificam algumas formas de como criar e utilizar as matrizes de visão e de projeção.

<table>
    <tr>
        <td> Projeção perspectiva <img src='cg/images/ex8_solar_system_5.png' style="width:400px"> </td>
        <td> Projeção ortográfica <img src='cg/images/ex8_solar_system_6.png' style="width:400px"></td>
    </tr>
</table>

## Resolução - Parte 2 e 3

In [17]:
import glm
import time
import math
import numpy as np
import OpenGL.GL as gl
from PyQt5 import QtOpenGL, QtCore
from PyQt5.QtCore import QCoreApplication
from PyQt5.QtWidgets import QApplication

from cg.shader_programs.SimpleShaderProgram_v2 import SimpleShaderProgram
from cg.renderers.ModelRenderer_v2 import ModelRenderer
from cg.models.SphereMesh_v1 import SphereMesh
from cg.models.OrbitPolygon import OrbitPolygon
from cg.models.SquareMesh_v1 import SquareMesh

AU = 1.496E8
planet_scale_factor = 1500.0
moon_scale_factor = planet_scale_factor * 0.05
sun_scale_factor = 50

# classe para armazenar as informações do Sol
class SunInfo:
    def __init__(self, color, diameter):
        
        self.color = color
        self.diameter = (diameter / AU) * sun_scale_factor
        self.radius = self.diameter / 2.0

# classe para armazenar as informações dos planetas
class PlanetInfo:
    def __init__(self, color, diameter, orbitalPeriod, orbitalInclination, orbitSemiMajorAxis, orbitSemiMinorAxis, orbitFocus):
        
        earth_days = 365.2
        
        self.color = color
        self.diameter = (diameter / AU) * planet_scale_factor
        self.radius = self.diameter / 2.0
        self.velocity = earth_days / orbitalPeriod
        
        self.orbitalPeriod = orbitalPeriod
        self.orbitalInclination = orbitalInclination 
        self.orbitSemiMajorAxis = orbitSemiMajorAxis
        self.orbitSemiMinorAxis = orbitSemiMinorAxis
        self.orbitFocus = orbitFocus

# cria um dicionário contendo as informações dos planetas       
PlanetarySheet = {
       'sun': SunInfo(np.array([1.00, 1.00, 0.00, 1.0]), 1391900),
       'earth': PlanetInfo(np.array([0.61, 0.79, 0.37, 1.0]), 12742, 365.2, 1.57, 1.0027, 1.0025, 0.0167),
       'moon': PlanetInfo(np.array([0.50, 0.50, 0.50, 1.0]), 3475, 27.3, 5.1, 0.0025718 * moon_scale_factor, 0.0021479 * moon_scale_factor, 0.00014537),
       'jupiter': PlanetInfo(np.array([0.83, 0.67, 0.53, 1.0]), 142984, 4331, 0.32, 5.2073, 5.2010, 0.2520)
       }

# classe para armazenar as informações gráficas em um único objeto
class GraphicObject:
    def __init__(self):
        self.model = None
        self.renderer = None
        self.modelMatrix = glm.mat4()
        self.color = np.array([1.0, 1.0, 1.0, 1.0])
        
class MyWidget(QtOpenGL.QGLWidget):

    def initializeGL(self):
        
        self.planetName = ['earth', 'moon', 'jupiter',]
        self.orbits = {}
        self.planets = {}
        
        # cria a matriz de visão
        self.cameraPos   = glm.vec3(0.0, 0.0, 15.0)
        self.cameraFront = glm.vec3(0.0, 0.0, -1.0)
        self.cameraUp    = glm.vec3(0.0, 1.0,  0.0)
        self.viewMatrix = glm.lookAt(self.cameraPos, self.cameraFront, self.cameraUp)
        
        #cria a matriz de projeção ortogonal
        self.orthoMatrix = glm.ortho(-20, 20, -20, 20, 0, 100)
        
        # define uma transformação global
        self.globalTransform = glm.mat4()
        
        # cria um shader program simples   
        # ativa o shader programa para permitir configurar uma cor única para todos os vértices
        self.shaderProgram = SimpleShaderProgram()
        self.shaderProgram.bind()
        self.shaderProgram.useUniformColor(True)
        self.shaderProgram.release()
        
        position_loc = self.shaderProgram.getVertexPositionLoc()

        # cria uma malhar de triângulos esférica comum para objetos esféricos
        sphere_mesh = SphereMesh(1.0, 30, 30)
        sphere_renderer = ModelRenderer(sphere_mesh.getVertexPositions(), vertex_indices=sphere_mesh.getVertexIndices())
        sphere_renderer.setVertexPositionLoc(position_loc)
        
        # laço para cria os objetos onde as informações gráficas das órbita são armazenadas
        for name in self.planetName:
            planet_info = PlanetarySheet[name]
            self.orbits[name] = GraphicObject()
            self.orbits[name].model = OrbitPolygon(planet_info.orbitSemiMajorAxis, planet_info.orbitSemiMinorAxis, planet_info.orbitFocus, 50)
            self.orbits[name].renderer = ModelRenderer(self.orbits[name].model.getVertexPositions(), primitive=ModelRenderer.LINE_LOOP)
            self.orbits[name].renderer.setVertexPositionLoc(position_loc)
            self.orbits[name].color = planet_info.color
        
        # laço para cria os objeto onde as informações gráficas dos planetas são armazenadas
        for name in self.planetName:
            self.planets[name] = GraphicObject()
            self.planets[name].model = sphere_mesh
            self.planets[name].renderer = sphere_renderer
            self.planets[name].color = PlanetarySheet[name].color
        
        # cria um objeto para armazenar as informações gráficas do Sol
        self.sun = GraphicObject()
        self.sun.model = sphere_mesh
        self.sun.renderer = sphere_renderer
        self.sun.color = PlanetarySheet['sun'].color
        self.sun.modelMatrix = glm.scale(glm.mat4(), glm.vec3(PlanetarySheet['sun'].radius, PlanetarySheet['sun'].radius, PlanetarySheet['sun'].radius))
        
        # cria um objeto para armazenar as informações do quadrado azul
        self.square = GraphicObject()
        self.square.model = SquareMesh(25, 25, 10, 10)
        self.square.renderer = ModelRenderer(self.square.model.getVertexPositions(), vertex_indices=self.square.model.getVertexIndices())
        self.square.renderer.setVertexPositionLoc(position_loc)
        self.square.color = np.array([0.0, 0.0, 0.2, 1.0])
        self.square.modelMatrix = glm.translate(glm.mat4(), glm.vec3(0.0, 0.0, -15.0))
        
        # cria um objeto para armazenar as informações do quadrado verde
        self.square2 = GraphicObject()
        self.square2.model = SquareMesh(30, 30, 10, 10)
        self.square2.renderer = ModelRenderer(self.square2.model.getVertexPositions(), vertex_indices=self.square2.model.getVertexIndices())
        self.square2.renderer.setVertexPositionLoc(position_loc)
        self.square2.color = np.array([0.0, 0.6, 0.0, 1.0])
        self.square2.modelMatrix = glm.translate(glm.mat4(), glm.vec3(0.0, 0.0, -20.0))
        
        # armazena o momento que o programa começou
        self.startTime = time.time()
    
    def paintGL(self):
        
        # configura a cor de background
        gl.glClearColor(1, 1, 1, 1)
        
        # limpa o background com a cor especificada
        gl.glClear(gl.GL_COLOR_BUFFER_BIT | gl.GL_DEPTH_BUFFER_BIT)
        
        # ativa o teste de profunidade e o recurso de culling
        gl.glEnable(gl.GL_DEPTH_TEST);
        gl.glEnable(gl.GL_CULL_FACE)
        
        # calcula o tempo de execução do programa
        self.currentTime = time.time()
        time_elapsed = self.currentTime - self.startTime
        
        # atualiza a posição dos objetos gráficos
        self.updatePositions(time_elapsed)
        
        # ativa o shader program que será executado pela GPU
        self.shaderProgram.bind()
        
        # atualiza o viewport para renderizar no lado esquerdo
        gl.glViewport(0, 0, int(self.width / 2), self.height)
        
        # troca a matrix de projeção
        self.projectionMatrix = self.perspectiveMatrix
        
        # renderiza os objetos
        self.renderObject(self.square2)
        self.renderObject(self.square)
        self.renderObject(self.sun)
        self.renderPlanets()        
        self.renderOrbits()
        
        # atualiza o viewport para renderizar no lado direito
        gl.glViewport(int(self.width / 2), 0, int(self.width / 2), self.height)
        
        # troca a matrix de projeção
        self.projectionMatrix = self.orthoMatrix
        
        # renderiza os objetos
        self.renderObject(self.square2)
        self.renderObject(self.square)
        self.renderObject(self.sun)
        self.renderPlanets()        
        self.renderOrbits()
        
        # desativa o shader program
        self.shaderProgram.release()
        
        # solicita que o método paintGL seja chamado novamente
        self.update()
    
    def resizeGL(self, width, height):
        
        # armazena a largura e altura da janela
        self.width = width
        self.height = height
        
        # atualiza a matriz de projeção perspectiva
        aspectRatio = int(self.width / 2) / height
        self.perspectiveMatrix = glm.perspective(glm.radians(60.0), aspectRatio, 0.5, 100.0)
    
    def updatePositions(self, time_elapsed):
        
        angle = time_elapsed * 2 * math.pi * 0.05

        self.updatePlanetPosition(angle)
        self.updateMoonOrbitPosition(angle)
        self.updateMoonPosition(angle)
        
    def updatePlanetPosition(self, angle):
        
        # atualiza a transformação de todos os planetas
        for name in self.planetName:
            
            # recupera a posição do planeta
            # e cria uma matriz de translação para levar o planeta para a nova posição
            planet_info = PlanetarySheet[name]
            x, y = self.orbits[name].model.getPositionAt(angle * planet_info.velocity)
            transl_planet_pos = glm.translate(glm.mat4(), glm.vec3(x, y, 0.0))
            
            # cria uma matriz de escala para deixar o planeta com o tamanho correto
            scale_planet_size = glm.scale(glm.mat4(), glm.vec3(planet_info.radius, planet_info.radius, planet_info.radius))
            
            # calcula a matriz de transformação final do planeta
            self.planets[name].modelMatrix = transl_planet_pos * scale_planet_size
        
    def updateMoonOrbitPosition(self, angle):
        
        # recupera a posição da Terra na órbita
        # e cria uma matriz de translação para levar a órbita da lua para a posição da Terra
        x, y = self.orbits['earth'].model.getPositionAt(angle * PlanetarySheet['earth'].velocity)
        trans_earth_pos = glm.translate(glm.mat4(), glm.vec3(x, y, 0.0))
        
        # calcula a matriz de transformação final da órbita da lua
        self.orbits['moon'].modelMatrix = trans_earth_pos
    
    def updateMoonPosition(self, angle):
        
        # recupera a posição da lua na órbita
        # e cria uma matriz de translação para levar a lua para a nova posição
        x, y = self.orbits['moon'].model.getPositionAt(angle * PlanetarySheet['moon'].velocity)
        transl_moon_pos = glm.translate(glm.mat4(), glm.vec3(x, y, 0.0))
        
        # recupera a posição da Terra na órbita
        # e cria uma matriz de translação para levar a lua para a posição da Terra
        x, y = self.orbits['earth'].model.getPositionAt(angle * PlanetarySheet['earth'].velocity)
        transl_earth_pos = glm.translate(glm.mat4(), glm.vec3(x, y, 0.0))
        
        # cria uma matriz de escala para deixar a lua com o tamanho correto
        scale_moon_size = glm.scale(glm.mat4(), glm.vec3(PlanetarySheet['moon'].radius, PlanetarySheet['moon'].radius, PlanetarySheet['moon'].radius))

        # calcula a matriz de transformação final da lua
        self.planets['moon'].modelMatrix = transl_earth_pos * transl_moon_pos * scale_moon_size
        
    def renderObject(self, graphic_object):
        
        # atualiza o shader com a transformação dos objetos
        self.shaderProgram.setUniformMVPMatrix(self.projectionMatrix * self.viewMatrix * self.globalTransform * graphic_object.modelMatrix)
        self.shaderProgram.setUniformColor(graphic_object.color)
        
        # renderiza o objeto
        graphic_object.renderer.render()
        
    def renderPlanets(self):
        
        # laço para renderizar todos os planetas
        for name in self.planetName:
            self.renderObject(self.planets[name])
            
    def renderOrbits(self):
        
        # laço para renderizar todas as órbitas
        for name in self.planetName:
            self.renderObject(self.orbits[name])      
    
    def keyPressEvent(self, event):
        super(MyWidget, self).keyPressEvent(event)
        
        # passo 
        step = 0.5
        
        # verifica se foi pressionada a tecla de seta 'para cima'
        if event.key() == QtCore.Qt.Key_Up:
            self.cameraPos.y += step

        # verifica se foi pressionada a tecla de seta 'para baixo'
        elif event.key() == QtCore.Qt.Key_Down:
            self.cameraPos.y -= step
        
        # verifica se foi pressionada a tecla de seta 'para esquerda'
        elif event.key() == QtCore.Qt.Key_Left:
            self.cameraPos.x -= step
        
        # verifica se foi pressionada a tecla de seta 'para direita'
        elif event.key() == QtCore.Qt.Key_Right:
            self.cameraPos.x += step
        
        # verifica se foi pressionada a tecla de seta '-'
        elif event.key() == QtCore.Qt.Key_Minus:
            self.cameraPos.z -= step
        
        # verifica se foi pressionada a tecla de seta '+'
        elif event.key() == QtCore.Qt.Key_Plus:
            self.cameraPos.z += step
        
        # atualiza a matriz de visão e global com os novos parâmetros
        self.viewMatrix = glm.lookAt(self.cameraPos, self.cameraFront, self.cameraUp)
        
def main():
    import sys

    app = QCoreApplication.instance()
    if app is None:
        app = QApplication(sys.argv)

    glformat = QtOpenGL.QGLFormat()
    glformat.setVersion(3, 3)
    glformat.setProfile(QtOpenGL.QGLFormat.CoreProfile)
    
    w = MyWidget(glformat)
    w.resize(1000, 500)
    w.setWindowTitle('Solar System')
    w.show()
    
    output = app.exec_()
    sys.exit(output)
    
if __name__ == '__main__':
    main()

SystemExit: 0

C:\Users\Rapha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Parte 4 - Completando o sistema solar

Complete o sistema solar adicionando os planetas que estão faltando. O código abaixo fornece os dados necessários para fazer isso. Além disso, altere o tamanho dos quadrados de fundo. Para o quadrado que está mais perto (na posição (0, 0, -15)), altere o tamanho para 130x130. Para o quadrado mais distante, altere o tamanho para 150x150. Por fim, configure as matrizes de visão e de projeção perspectiva para gerar a imagem abaixo. Os notebooks ([29_Transformacao_visao](29_Transformacao_visao.ipynb)) e ([31_Transformacao_projecao_perspectiva](31_Transformacao_projecao_perspectiva.ipynb)) exemplificam algumas formas de como criar e utilizar as matrizes de visão e de projeção.

<td><img src='cg/images/ex8_solar_system_7.png' style="width:400px">

In [25]:
import glm
import time
import math
import numpy as np
import OpenGL.GL as gl
from PyQt5 import QtOpenGL, QtCore
from PyQt5.QtCore import QCoreApplication
from PyQt5.QtWidgets import QApplication

from cg.shader_programs.SimpleShaderProgram_v2 import SimpleShaderProgram
from cg.renderers.ModelRenderer_v2 import ModelRenderer
from cg.models.SphereMesh_v1 import SphereMesh
from cg.models.OrbitPolygon import OrbitPolygon
from cg.models.SquareMesh_v1 import SquareMesh

AU = 1.496E8
planet_scale_factor = 1500.0
moon_scale_factor = planet_scale_factor * 0.05
sun_scale_factor = 50

# classe para armazenar as informações do Sol
class SunInfo:
    def __init__(self, color, diameter):
        
        self.color = color
        self.diameter = (diameter / AU) * sun_scale_factor
        self.radius = self.diameter / 2.0

# classe para armazenar as informações dos planetas
class PlanetInfo:
    def __init__(self, color, diameter, orbitalPeriod, orbitalInclination, orbitSemiMajorAxis, orbitSemiMinorAxis, orbitFocus):
        
        earth_days = 365.2
        
        self.color = color
        self.diameter = (diameter / AU) * planet_scale_factor
        self.radius = self.diameter / 2.0
        self.velocity = earth_days / orbitalPeriod
        
        self.orbitalPeriod = orbitalPeriod
        self.orbitalInclination = orbitalInclination 
        self.orbitSemiMajorAxis = orbitSemiMajorAxis
        self.orbitSemiMinorAxis = orbitSemiMinorAxis
        self.orbitFocus = orbitFocus

# cria um dicionário contendo as informações dos planetas       
PlanetarySheet = {
       'sun': SunInfo(np.array([1.00, 1.00, 0.00, 1.0]), 1391900),
       'mercury': PlanetInfo(np.array([0.96, 0.90, 0.71, 1.0]), 4866, 88.0, 6.34, 0.3870, 0.3788, 0.0796),
       'venus': PlanetInfo(np.array([0.95, 0.82, 0.38, 1.0]), 12106, 224.7, 2.19, 0.7219, 0.7219, 0.0049),
       'earth': PlanetInfo(np.array([0.61, 0.79, 0.37, 1.0]), 12742, 365.2, 1.57, 1.0027, 1.0025, 0.0167),
       'moon': PlanetInfo(np.array([0.50, 0.50, 0.50, 1.0]), 3475, 27.3, 5.1, 0.0025718 * moon_scale_factor, 0.0021479 * moon_scale_factor, 0.00014537),
       'mars': PlanetInfo(np.array([0.88, 0.81, 0.61, 1.0]), 6760, 687.0, 1.67, 1.5241, 1.5173, 0.1424),
       'jupiter': PlanetInfo(np.array([0.83, 0.67, 0.53, 1.0]), 142984, 4331, 0.32, 5.2073, 5.2010, 0.2520),
       'saturn': PlanetInfo(np.array([0.89, 0.87, 0.63, 1.0]), 116438, 10747, 0.93, 9.5590, 9.5231, 0.5181),
       'uranus': PlanetInfo(np.array([0.00, 0.87, 0.95, 1.0]), 46940, 30589, 1.02, 19.1848, 19.1645, 0.9055),
       'neptune': PlanetInfo(np.array([0.00, 0.51, 0.89, 1.0]), 45432, 59800, 0.72, 30.0806, 30.0788, 0.2587),
       'pluto': PlanetInfo(np.array([0.62, 0.63, 0.64, 1.0]), 2274, 90560, 15.55, 39.5, 34.031, 9.8276)
       }

# classe para armazenar as informações gráficas em um único objeto
class GraphicObject:
    def __init__(self):
        self.model = None
        self.renderer = None
        self.modelMatrix = glm.mat4()
        self.color = np.array([1.0, 1.0, 1.0, 1.0])
        
class MyWidget(QtOpenGL.QGLWidget):

    def initializeGL(self):
        
        self.planetName = ['mercury', 'venus', 'earth', 'moon', 'mars', 'jupiter', 'saturn', 'uranus', 'neptune', 'pluto']
        self.orbits = {}
        self.planets = {}
        
        # cria a matriz de visão
        self.cameraPos   = glm.vec3(0.0,-61.5, 87)
        self.cameraFront = glm.vec3(0.0, 0.0, -1.0)
        self.cameraUp    = glm.vec3(0.0, 1.0,  0.0)
        self.viewMatrix = glm.lookAt(self.cameraPos, self.cameraFront, self.cameraUp)
        
        # define uma transformação global
        self.globalTransform = glm.mat4()
        
        # cria um shader program simples   
        # ativa o shader programa para permitir configurar uma cor única para todos os vértices
        self.shaderProgram = SimpleShaderProgram()
        self.shaderProgram.bind()
        self.shaderProgram.useUniformColor(True)
        self.shaderProgram.release()
        
        position_loc = self.shaderProgram.getVertexPositionLoc()

        # cria uma malhar de triângulos esférica comum para objetos esféricos
        sphere_mesh = SphereMesh(1.0, 30, 30)
        sphere_renderer = ModelRenderer(sphere_mesh.getVertexPositions(), vertex_indices=sphere_mesh.getVertexIndices())
        sphere_renderer.setVertexPositionLoc(position_loc)
        
        # laço para cria os objetos onde as informações gráficas das órbita são armazenadas
        for name in self.planetName:
            planet_info = PlanetarySheet[name]
            self.orbits[name] = GraphicObject()
            self.orbits[name].model = OrbitPolygon(planet_info.orbitSemiMajorAxis, planet_info.orbitSemiMinorAxis, planet_info.orbitFocus, 50)
            self.orbits[name].renderer = ModelRenderer(self.orbits[name].model.getVertexPositions(), primitive=ModelRenderer.LINE_LOOP)
            self.orbits[name].renderer.setVertexPositionLoc(position_loc)
            self.orbits[name].color = planet_info.color
        
        # laço para cria os objeto onde as informações gráficas dos planetas são armazenadas
        for name in self.planetName:
            self.planets[name] = GraphicObject()
            self.planets[name].model = sphere_mesh
            self.planets[name].renderer = sphere_renderer
            self.planets[name].color = PlanetarySheet[name].color
        
        # cria um objeto para armazenar as informações gráficas do Sol
        self.sun = GraphicObject()
        self.sun.model = sphere_mesh
        self.sun.renderer = sphere_renderer
        self.sun.color = PlanetarySheet['sun'].color
        self.sun.modelMatrix = glm.scale(glm.mat4(), glm.vec3(PlanetarySheet['sun'].radius, PlanetarySheet['sun'].radius, PlanetarySheet['sun'].radius))
        
        # cria um objeto para armazenar as informações do quadrado azul
        self.square = GraphicObject()
        self.square.model = SquareMesh(130, 130, 10, 10)
        self.square.renderer = ModelRenderer(self.square.model.getVertexPositions(), vertex_indices=self.square.model.getVertexIndices())
        self.square.renderer.setVertexPositionLoc(position_loc)
        self.square.color = np.array([0.0, 0.0, 0.2, 1.0])
        self.square.modelMatrix = glm.translate(glm.mat4(), glm.vec3(0.0, 0.0, -15.0))
        
        # cria um objeto para armazenar as informações do quadrado verde
        self.square2 = GraphicObject()
        self.square2.model = SquareMesh(150, 150, 10, 10)
        self.square2.renderer = ModelRenderer(self.square2.model.getVertexPositions(), vertex_indices=self.square2.model.getVertexIndices())
        self.square2.renderer.setVertexPositionLoc(position_loc)
        self.square2.color = np.array([0.0, 0.6, 0.0, 1.0])
        self.square2.modelMatrix = glm.translate(glm.mat4(), glm.vec3(0.0, 0.0, -20.0))
        
        # armazena o momento que o programa começou
        self.startTime = time.time()
    
    def paintGL(self):
        
        # configura a cor de background
        gl.glClearColor(1, 1, 1, 1)
        
        # limpa o background com a cor especificada
        gl.glClear(gl.GL_COLOR_BUFFER_BIT | gl.GL_DEPTH_BUFFER_BIT)
        
        # ativa o teste de profunidade e o recurso de culling
        gl.glEnable(gl.GL_DEPTH_TEST);
        gl.glEnable(gl.GL_CULL_FACE)
        
        # calcula o tempo de execução do programa
        self.currentTime = time.time()
        time_elapsed = self.currentTime - self.startTime
        
        # atualiza a posição dos objetos gráficos
        self.updatePositions(time_elapsed)
        
        # ativa o shader program que será executado pela GPU
        self.shaderProgram.bind()
        
        # renderiza os objetos
        self.renderObject(self.square2)
        self.renderObject(self.square)
        self.renderObject(self.sun)
        self.renderPlanets()        
        self.renderOrbits()
        
        # desativa o shader program
        self.shaderProgram.release()
        
        # solicita que o método paintGL seja chamado novamente
        self.update()
    
    def resizeGL(self, width, height):
        
        # atualiza o viewport
        gl.glViewport(0, 0, width, height)
        
        # atualiza a matriz de projeção perspectiva
        aspectRatio = width / height
        self.perspectiveMatrix = glm.perspective(45.0, aspectRatio, 0.1, 200.0)
    
    def updatePositions(self, time_elapsed):
        
        angle = time_elapsed * 2 * math.pi * 0.05

        self.updatePlanetPosition(angle)
        self.updateMoonOrbitPosition(angle)
        self.updateMoonPosition(angle)
        
    def updatePlanetPosition(self, angle):
        
        # atualiza a transformação de todos os planetas
        for name in self.planetName:
            
            # recupera a posição do planeta
            # e cria uma matriz de translação para levar o planeta para a nova posição
            planet_info = PlanetarySheet[name]
            x, y = self.orbits[name].model.getPositionAt(angle * planet_info.velocity)
            transl_planet_pos = glm.translate(glm.mat4(), glm.vec3(x, y, 0.0))
            
            # cria uma matriz de escala para deixar o planeta com o tamanho correto
            scale_planet_size = glm.scale(glm.mat4(), glm.vec3(planet_info.radius, planet_info.radius, planet_info.radius))
            
            # calcula a matriz de transformação final do planeta
            self.planets[name].modelMatrix = transl_planet_pos * scale_planet_size
        
    def updateMoonOrbitPosition(self, angle):
        
        # recupera a posição da Terra na órbita
        # e cria uma matriz de translação para levar a órbita da lua para a posição da Terra
        x, y = self.orbits['earth'].model.getPositionAt(angle * PlanetarySheet['earth'].velocity)
        trans_earth_pos = glm.translate(glm.mat4(), glm.vec3(x, y, 0.0))
        
        # calcula a matriz de transformação final da órbita da lua
        self.orbits['moon'].modelMatrix = trans_earth_pos
    
    def updateMoonPosition(self, angle):
        
        # recupera a posição da lua na órbita
        # e cria uma matriz de translação para levar a lua para a nova posição
        x, y = self.orbits['moon'].model.getPositionAt(angle * PlanetarySheet['moon'].velocity)
        transl_moon_pos = glm.translate(glm.mat4(), glm.vec3(x, y, 0.0))
        
        # recupera a posição da Terra na órbita
        # e cria uma matriz de translação para levar a lua para a posição da Terra
        x, y = self.orbits['earth'].model.getPositionAt(angle * PlanetarySheet['earth'].velocity)
        transl_earth_pos = glm.translate(glm.mat4(), glm.vec3(x, y, 0.0))
        
        # cria uma matriz de escala para deixar a lua com o tamanho correto
        scale_moon_size = glm.scale(glm.mat4(), glm.vec3(PlanetarySheet['moon'].radius, PlanetarySheet['moon'].radius, PlanetarySheet['moon'].radius))

        # calcula a matriz de transformação final da lua
        self.planets['moon'].modelMatrix = transl_earth_pos * transl_moon_pos * scale_moon_size
        
    def renderObject(self, graphic_object):
        
        # atualiza o shader com a transformação dos objetos
        self.shaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * self.viewMatrix * self.globalTransform * graphic_object.modelMatrix)
        self.shaderProgram.setUniformColor(graphic_object.color)
        
        # renderiza o objeto
        graphic_object.renderer.render()
        
    def renderPlanets(self):
        
        # laço para renderizar todos os planetas
        for name in self.planetName:
            self.renderObject(self.planets[name])
            
    def renderOrbits(self):
        
        # laço para renderizar todas as órbitas
        for name in self.planetName:
            self.renderObject(self.orbits[name])      
    
    def keyPressEvent(self, event):
        super(MyWidget, self).keyPressEvent(event)
        
        # passo 
        step = 0.5
        
        # verifica se foi pressionada a tecla de seta 'para cima'
        if event.key() == QtCore.Qt.Key_Up:
            self.cameraPos.y += step

        # verifica se foi pressionada a tecla de seta 'para baixo'
        elif event.key() == QtCore.Qt.Key_Down:
            self.cameraPos.y -= step
        
        # verifica se foi pressionada a tecla de seta 'para esquerda'
        elif event.key() == QtCore.Qt.Key_Left:
            self.cameraPos.x -= step
        
        # verifica se foi pressionada a tecla de seta 'para direita'
        elif event.key() == QtCore.Qt.Key_Right:
            self.cameraPos.x += step
        
        # verifica se foi pressionada a tecla de seta '-'
        elif event.key() == QtCore.Qt.Key_Minus:
            self.cameraPos.z -= step
        
        # verifica se foi pressionada a tecla de seta '+'
        elif event.key() == QtCore.Qt.Key_Plus:
            self.cameraPos.z += step
        
        # atualiza a matriz de visão e global com os novos parâmetros
        self.viewMatrix = glm.lookAt(self.cameraPos, self.cameraFront, self.cameraUp)
        
def main():
    import sys

    app = QCoreApplication.instance()
    if app is None:
        app = QApplication(sys.argv)

    glformat = QtOpenGL.QGLFormat()
    glformat.setVersion(3, 3)
    glformat.setProfile(QtOpenGL.QGLFormat.CoreProfile)
    
    w = MyWidget(glformat)
    w.resize(800, 800)
    w.setWindowTitle('Solar System')
    w.show()
    
    output = app.exec_()
    sys.exit(output)
    
if __name__ == '__main__':
    main()

SystemExit: 0

C:\Users\Rapha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
